### --- Day 21: Step Counter ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/21

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 21);
WriteLines(inputLines);

Loading puzzle file: Day21.txt
Total lines: 131
Max line length: 131

...................................................................................................................................
...................#.....#.....#.#.............#.#......#.......................#......#.......#......#...#...#.##..........##.....
......#......#.....#....#........#...............#..........................#................#...........#...#......#.....#.##.#...
..#.........#......#........##..##.#............#.#.#.#...................##....#......#.#............#...........#................
.#...............#....###.#....#...........#........##..............................#...............##.............#....#..........


In [4]:
const char GardenPlot = '.';
const char Rock = '#';
const char StartingSquare = 'S';

record Point(int row, int col) {
    Point North => new(row - 1, col);
    Point South => new(row + 1, col);
    Point East => new(row, col - 1);
    Point West => new(row, col + 1);

    public Point[] All => [North, South, East, West];
}
record PlotPoint(Point point, char plotType);

static Dictionary<Point, PlotPoint> ParseGarden(string[] map) {
    var q = from row in Enumerable.Range(0, map.Length)
            from col in Enumerable.Range(0, map[0].Length)
            let point = new Point(row, col)
            select new PlotPoint(point, map[row][col]);

    return q.ToDictionary(p => p.point);
}

In [5]:
class Garden 
{
    readonly Dictionary<Point, PlotPoint> gardenPoints;
    readonly int rows;
    readonly int cols;
    readonly Point startPoint;

    public Garden(string[] map) {
        gardenPoints = ParseGarden(map);

        rows = gardenPoints.Values.Max(gp => gp.point.row) + 1;
        cols = gardenPoints.Values.Max(gp => gp.point.col) + 1;
        startPoint = gardenPoints.Values.Single(gp => gp.plotType == StartingSquare).point;
    }

    public HashSet<Point> Explore(int steps)
    {
        HashSet<Point> currentPoints = [startPoint];

        for (var i = 0; i < steps; i++)
        {
            var nextPoints = currentPoints.SelectMany(cp => cp.All).Distinct().ToList();

            var q = from p in nextPoints
            where 
                p.row >= 0
                && p.row < rows
                && p.col >= 0
                && p.col < cols
            let gardenPoint = gardenPoints[p]
            where gardenPoint.plotType != Rock
            select p;

            currentPoints.Clear();
            currentPoints.UnionWith(q);
        }

        return currentPoints;
    }

    public string Render(HashSet<Point> reachablePoints = null)
    {
        reachablePoints ??= [];

        var sb = new StringBuilder();
        foreach (var row in Enumerable.Range(0, rows)) {
            foreach (var col in Enumerable.Range(0, cols)) {
                Point p = new(row, col);
                if (reachablePoints.Contains(p)) {
                    sb.Append("O");
                } else {
                    sb.Append(gardenPoints[p].plotType);
                }
            }
            sb.AppendLine();
        }
        return sb.ToString();
    }
}

In [6]:
string[] testInputLines = [
    "...........",
    ".....###.#.",
    ".###.##..#.",
    "..#.#...#..",
    "....#.#....",
    ".##..S####.",
    ".##..#...#.",
    ".......##..",
    ".##.#.####.",
    ".##..##.##.",
    "...........",
];

In [7]:
var testGarden = new Garden(testInputLines);
Console.WriteLine(testGarden.Render());

...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........



In [8]:
// After a total of 6 steps, he could reach any of the garden plots marked O:

// ...........
// .....###.#.
// .###.##.O#.
// .O#O#O.O#..
// O.O.#.#.O..
// .##O.O####.
// .##.O#O..#.
// .O.O.O.##..
// .##.#.####.
// .##O.##.##.
// ...........

var testReachable = testGarden.Explore(6);
Console.WriteLine(testGarden.Render(testReachable));

...........
.....###.#.
.###.##.O#.
.O#O#O.O#..
O.O.#.#.O..
.##O.O####.
.##.O#O..#.
.O.O.O.##..
.##.#.####.
.##O.##.##.
...........



In [9]:
// Starting from the garden plot marked S on your map, how many garden plots could the Elf reach in exactly 64 steps?

var part1Garden = new Garden(inputLines);
var part1Answer = part1Garden.Explore(64).Count;
Console.WriteLine(part1Answer);

3764


In [10]:
// 3764 is correct!
Ensure(3764, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/21

In [12]:
// Ok so we're going to need an infinte tile garden simulator

In [13]:
class Garden2
{
    readonly Dictionary<Point, PlotPoint> gardenPoints;
    readonly int rows;
    readonly int cols;
    readonly Point startPoint;

    public Garden2(string[] map) {
        gardenPoints = ParseGarden(map);

        rows = cols = map.Length; // assume square
        startPoint = gardenPoints.Values.Single(gp => gp.plotType == StartingSquare).point;
    }

    PlotPoint GetPoint(Point p)
    {
        var (row, col) = (p.row % rows, p.col % cols);

        row = row < 0 ? row + rows : row;
        col = col < 0 ? col + cols : col;

        Point mappedPoint = new(row, col);

        return gardenPoints[mappedPoint];
    }

    public HashSet<Point> Explore(int steps)
    {
        HashSet<Point> discovered = [startPoint];
        HashSet<Point> currentPoints = [startPoint];

        var i = steps % 2;
        if (i == 1) {
            discovered.Clear();
            currentPoints.Clear();
            discovered.UnionWith(startPoint.All);
            currentPoints.UnionWith(discovered);
        }

        for (; i < steps; i += 2)
        {
            HashSet<Point> nextPoints = new();
            foreach (var cp in currentPoints) {
                foreach (var cpNext in cp.All) {
                    var gardenPoint = GetPoint(cpNext);
                    if (gardenPoint.plotType == Rock) continue;

                    foreach (var cp2 in cpNext.All) {
                        if (discovered.Contains(cp2)) continue;
                        
                        gardenPoint = GetPoint(cp2);
                        if (gardenPoint.plotType == Rock) continue;

                        nextPoints.Add(cp2);
                    }
                }
            }

            currentPoints.Clear();
            currentPoints.UnionWith(nextPoints);
            discovered.UnionWith(currentPoints);
        }

        return discovered;
    }

    public int ExploreCount(int steps) => Explore(steps).Count;

    public string Render(HashSet<Point> reachablePoints = null)
    {
        reachablePoints ??= [];

        var sb = new StringBuilder();
        foreach (var row in Enumerable.Range(0, rows)) {
            foreach (var col in Enumerable.Range(0, cols)) {
                Point p = new(row, col);
                if (reachablePoints.Contains(p)) {
                    sb.Append("O");
                } else {
                    sb.Append(gardenPoints[p].plotType);
                }
            }
            sb.AppendLine();
        }
        return sb.ToString();
    }
}

In [14]:
var testGarden2 = new Garden2(testInputLines);

// In exactly 6 steps, he can still reach 16 garden plots.
// In exactly 10 steps, he can reach any of 50 garden plots.
// In exactly 50 steps, he can reach 1594 garden plots.
// In exactly 100 steps, he can reach 6536 garden plots.
// In exactly 500 steps, he can reach 167004 garden plots.
// In exactly 1000 steps, he can reach 668697 garden plots.
// In exactly 5000 steps, he can reach 16733044 garden plots.

Console.WriteLine(testGarden2.ExploreCount(6));
Console.WriteLine(testGarden2.ExploreCount(50));
Console.WriteLine(testGarden2.ExploreCount(100));
Console.WriteLine(testGarden2.ExploreCount(500));
Console.WriteLine(testGarden2.ExploreCount(1000));
// This one takes ~40 seconds
// Console.WriteLine(testGarden2.ExploreCount(5000));


16
1594
6536
167004
668697


In [15]:
// As suspected we can't just calculate our way to the result.

In [16]:
// Let's see if the diamond theory is correct. Essentially we can reach every second tile in the up, down, left, right directions. 

var expInputLines = File.ReadAllLines("Day21-exp.txt");
var expInputMap = new Garden(expInputLines);

var expResult = expInputMap.Explore(9);
Console.WriteLine(expInputMap.Render(expResult));
Console.WriteLine(expResult.Count);

...............................
...............................
...............................
...............................
...............................
...............................
...............O...............
..............O.O..............
.............O.O.O.............
............O.O.O.O............
...........O.O.O.O.O...........
..........O.O.O.O.O.O..........
.........O.O.O.O.O.O.O.........
........O.O.O.O.O.O.O.O........
.......O.O.O.O.O.O.O.O.O.......
......O.O.O.O.OSO.O.O.O.O......
.......O.O.O.O.O.O.O.O.O.......
........O.O.O.O.O.O.O.O........
.........O.O.O.O.O.O.O.........
..........O.O.O.O.O.O..........
...........O.O.O.O.O...........
............O.O.O.O............
.............O.O.O.............
..............O.O..............
...............O...............
...............................
...............................
...............................
...............................
...............................
...............................

100


In [17]:
// Ok, so for a given number n, without any rocks the total reachable points is (n + 1)^2

// But how do we adjust for rocks??

In [18]:
// Curiously there is a full diamond at 65 steps with 0 rocks on the outside

// Total steps to explore: 26501365

// Our garden has 131 cols, the S is right in the middle at col 66
// Likewise, 131 rows, S is at row 66

// I have a theory that our exploring reaches this diamond

var cols = 131;
var min_col = 0;
var max_col = cols - 1;
var middle = cols / 2;
var steps = 26501365;

var right = (middle + steps);
var rightMod = right % cols;

Console.WriteLine($"Display range: {min_col} ... {middle} ... {max_col}");
Console.WriteLine($"Position after {steps} steps: {right} = {rightMod}");

// So now we know that...
// The explored shape is a diamond, which terminates exactly on the edges of a tile

// The edges are not blocked by any rocks, so we always reach the full extremity of
// each edge- our reachable steps are not blocked by rocks on non-reachable steps

// Therefore the internal spaces blocked by rocks can be "walked around", leaving
// the only unreachable spaces are the reachable ones blocked by rocks



Display range: 0 ... 65 ... 130
Position after 26501365 steps: 26501430 = 130


In [19]:
// so once we explore 26501365 steps to the right, we are at one edge, and exploring 26501365 steps the other direction puts us at the other, so I guess our exploring is an even multiple of tiles:

(26501365 * 2 + 1) / 131

// = 404601, so yes it looks that way

404601

In [20]:
// Let's create a 3x3 off the original and render that?

// I think the rend result should be a number of full boxes plus triangles

string[] Replicate(string[] template, int times) {
    var result = new string[template.Length * times];
    var j = 0;
    foreach (var i in Enumerable.Range(0, times)) {
        foreach (var row in template) {
            var bigRow = Enumerable.Range(0, times).SelectMany(j => row.Replace('S', '.')).ToArray();
            var bigStr = new string(bigRow);
            result[j++] = bigStr;
        }
    }

    var middleRow = result[result.Length / 2].ToCharArray();
    middleRow[middleRow.Length / 2] = 'S';
    result[result.Length / 2] = new string(middleRow);

    return result;
}

In [21]:
Console.WriteLine(String.Join("\n", Replicate(inputLines, 3)));

.........................................................................................................................................................................................................................................................................................................................................................................................................
...................#.....#.....#.#.............#.#......#.......................#......#.......#......#...#...#.##..........##........................#.....#.....#.#.............#.#......#.......................#......#.......#......#...#...#.##..........##........................#.....#.....#.#.............#.#......#.......................#......#.......#......#...#...#.##..........##.....
......#......#.....#....#........#...............#..........................#................#...........#...#......#.....#.##.#.........#......#.....#....#........#...............#..........................#....

In [22]:
// Dammit I had to take a hint on this one :( Time to abandon the diamond idea.
// Repetition the theme but the key is to use tiles, not diamonds. There are only
// a certain number of unique tiles, and the big diamond can be composed of these

// Separate a replicated tile back into its individual tiles
IEnumerable<string> Separate(string[] template, string[] fullMap) {
    var rows = cols = template.Length; // square

    var times = fullMap.Length / template.Length;

    foreach (var i in Enumerable.Range(0, times)) {
        foreach (var j in Enumerable.Range(0, times)) {
            var startRow = i * template.Length;
            var startCol = j * template.Length;

            var tileResult = new string[template.Length];

            for (var row = 0; row < template.Length; row++) {
                tileResult[row] = fullMap[startRow + row].Substring(startCol, template.Length);
            }

            var tileStr = string.Join('\n', tileResult);
            yield return tileStr;
        }
    }
}

// Add the S back to the middle of each tile (helps recognise each tile on a large map)
string[] insertSS(string source, int tileSize) {
    var sourceLines = source.Split('\n');
    var result = new string[sourceLines.Length];

    var middle = tileSize / 2;

    for (var i = 0; i < sourceLines.Length; i++) {
        var newRow = sourceLines[i].ToCharArray();
        
        if (i % tileSize == middle) {
            for (var j = 0; j < newRow.Length; j++) {
                if (j % tileSize == middle) {
                    newRow[j] = 'S';
                }
            }
        }

        result[i] = (new string(newRow));
    }

    return result;
}

void SplitExplore(string[] template, int times) {
    var replicated = Replicate(template, times);

    var steps = ((template.Length * times) / 2);
    Console.WriteLine($"Steps for {times} times: {steps}");

    var garden = new Garden2(replicated);
    var explored = garden.Explore(steps);

    Console.WriteLine($"Reachable plots: {explored.Count}");

    var rendered = garden.Render(explored);

    var rendered_s = insertSS(rendered, template.Length);

    var filePath = $"Day21-SplitExplore-{times}.txt";

    Console.WriteLine($"Saving data to {filePath}");

    File.WriteAllLines(filePath, rendered_s);

    // Show the unique tiles and their counts
    // var tiles = Separate(inputLines, rendered_s).ToList();

    // var tileGroups = tiles.GroupBy(t => t).Select(tg => (tile: tg.Key, count: tg.Count()));
    // Console.WriteLine($"Total unique tiles: {tileGroups.Count()}");
    // foreach (var tg in tileGroups.OrderByDescending(tg => tg.count)) {
    //     Console.WriteLine($"Count: {tg.count}");
    //     Console.WriteLine(tg.tile);
    //     Console.WriteLine();
    // }
}

SplitExplore(inputLines, 5);

Steps for 5 times: 327
Reachable plots: 95442
Saving data to Day21-SplitExplore-5.txt


In [23]:
// From 5x5, we have 15 unique tiles

// For each row that is not the middle or the top, we have 2 end pieces- fortunately there're always the same!

// Therefore our unique tiles are (for 13x)...

// full OSO (25)
// full .S. (36)
// top middle x 2 (1 each)
// side middle x 2 (1 each)
// side piece inner x 4 (5 each)
// side piece outer x 4 (6 each)
// empty (60)

// so we need a minimum of 5x5, which is exploring 327 steps

var g5Map = Replicate(inputLines, 5);
var g5 = new Garden2(g5Map);
var g5Steps = 327;
var g5Result = g5.Explore(g5Steps);

Console.WriteLine(g5Result.Count());

int getTileExplored(string[] template, HashSet<Point> result, int tileRow, int tileCol)
{
    var tileSize = template.Length; // square

    var tileResult = result.Where(r => r.row / tileSize == tileRow 
                                    && r.col / tileSize == tileCol);
    return tileResult.Count();
}

var get = (int row, int col) => getTileExplored(inputLines, g5Result, row, col);

var fullOso = get(2, 2);
var fullAlt = get(2, 3);
var sideOuter = get(1, 0) + get (1, 4) + get(3, 0) + get(3, 4);
var sideInner = get(1, 1) + get (1, 3) + get(3, 1) + get(3, 3);
var topMiddle = get(0, 2) + get(4, 2);
var sideMiddle = get(2, 0) + get(2, 4);

ulong fullCount(int times) 
{
    checked {
        var outers = (ulong)times / 2;
        var inners = outers - 1;
        var fullOsoCount = inners * inners;
        var fullAltCount = outers * outers;

        ulong total = (ulong)topMiddle 
                    + (ulong)sideMiddle
                    + ((ulong)sideOuter * outers)
                    + ((ulong)sideInner * inners)
                    + ((ulong)fullOso * fullOsoCount)
                    + ((ulong)fullAlt * fullAltCount);

        return total;
    }
}

Console.WriteLine(fullCount(5)); // correct
Console.WriteLine(fullCount(9)); // correct
Console.WriteLine(fullCount(13)); // correct

95442
95442
308770
643866


In [24]:
// ...how many garden plots could the Elf reach in exactly 26501365 steps?

// As-stated above 26501365 steps convers 404601 tiles in width x height
var part2Answer = fullCount(404601);
Console.WriteLine(part2Answer);

622926941971282


In [25]:
// 622926941971282 is correct!
Ensure(622926941971282, part2Answer);